# Promedio Diario $PM_{2.5}$

Para nuestra transformación de datos, utilizaremos las siguientes librerias:


In [1]:
import pandas as pd
import numpy as np

- Cargando dataset a un DataFrame de pandas
- Posteriormente normalizando la columna `Fecha` a tipo `datetime`

In [2]:
#Cargando archivos
df = pd.read_csv('https://raw.githubusercontent.com/IrvingC48/BeduFase2-Proyecto_python/main/Datasets/PM2.5/PM25_depurado.csv')

# Normalizando fechas
df['FECHA'] = pd.to_datetime(df['FECHA'], unit='ns')
df.head()

,Unnamed: 0,FECHA,HORA,AJM,AJU,BJU,CAM,CCA,FAR,GAM,HGM,INN,MER,MON,NEZ,PED,SAC,SAG,SFE,TLA,UAX,UIZ,XAL
0,0,2019-01-01,1,19.0,35.0,62.0,90.0,66.0,NaN,NaN,56.0,22.0,69.0,102.0,133.0,NaN,NaN,92.0,33.0,55.0,74.0,99.0,96.0
1,1,2019-01-01,2,17.0,24.0,88.0,104.0,84.0,NaN,NaN,61.0,14.0,95.0,108.0,164.0,NaN,NaN,100.0,40.0,52.0,115.0,99.0,164.0
2,2,2019-01-01,3,14.0,20.0,107.0,140.0,95.0,NaN,NaN,74.0,8.0,82.0,125.0,206.0,NaN,NaN,104.0,52.0,59.0,150.0,109.0,249.0
3,3,2019-01-01,4,6.0,15.0,101.0,162.0,97.0,NaN,NaN,90.0,9.0,110.0,92.0,273.0,NaN,NaN,144.0,25.0,58.0,180.0,112.0,200.0
4,4,2019-01-01,5,4.0,8.0,121.0,133.0,88.0,NaN,NaN,90.0,3.0,165.0,84.0,291.0,NaN,NaN,171.0,21.0,46.0,167.0,164.0,161.0


- Reacomodo del DataFrame , donde ahora tendremos una columna para las estaciones y sus correspondientes datos de medición
- Eliminando los datos `NaN`
- Comprobando que no queden datos `NaN` en el DataFrame

In [3]:
# Extraer nombre de columnas
cols = df.columns.tolist()

# Reacomodando dataframe
df = pd.melt(df, id_vars=['FECHA', 'HORA'], value_vars=cols[3:], var_name='station', value_name='measurement')

# df = df[~df['measurement'].isna()]
df = df.dropna(axis=0, how='any').reset_index(drop=True)

print("No. de columnas que contienen valores nulos ")
print(len(df.columns[df.isna().any()]))

print("No. de columnas que no contienen valores nulos")
print(len(df.columns[df.notna().all()]))

print("No total de columnas en el dataframe")
print(len(df.columns))
df.head()

No. de columnas que contienen valores nulos 
0
No. de columnas que no contienen valores nulos
4
No total de columnas en el dataframe
4


,FECHA,HORA,station,measurement
0,2019-01-01,1,AJM,19.0
1,2019-01-01,2,AJM,17.0
2,2019-01-01,3,AJM,14.0
3,2019-01-01,4,AJM,6.0
4,2019-01-01,5,AJM,4.0


- Ordenando el dataframe segun la fecha y la estación

In [4]:
df = df.sort_values(['FECHA', 'station']).reset_index(drop=True)
df.head()

,FECHA,HORA,station,measurement
0,2019-01-01,1,AJM,19.0
1,2019-01-01,2,AJM,17.0
2,2019-01-01,3,AJM,14.0
3,2019-01-01,4,AJM,6.0
4,2019-01-01,5,AJM,4.0


Ahora se busca eliminar, según el día y la estación, aquellas filas que tengan mediciones horarias menores a las requeridas por la normativa (18 mediciones). Por ejemplo, si la estación ACO en el 01 de enero de 2019 registró solamente 15 mediciones horarias, todas las mediciones de ese día para la estación ACO no será tomadas en cuenta para los posteriores análisis. 

Primero se debe elaborar una función que devuelva el dataframe que contenga solamente los registros válidos

In [5]:
def dia_estacion_valido(dataframe):
  #Contar las mediciones horarias por día y estación
  filtro= dataframe.groupby(['FECHA', 'station'],as_index=False).agg(N_h=('HORA','count'))
  #Filtrar aquellas que no cumplen normativa
  filtro= filtro.query('N_h < 17')
  #Conservar las columnas de fecha y estación
  filtro=filtro.loc[:,['FECHA','station']]
  #Eliminar del dataframe original las mediciones que no cumplen normativa
  filtro=pd.merge(dataframe,filtro, how='outer', indicator=True)
  filtro=filtro[filtro['_merge'] == 'left_only']
  return filtro

Con la función, se filtra las mediciones por día y por estación son válidas para los siguientes análisis.

In [26]:
df_filtrado= dia_estacion_valido(df)
df_filtrado = df_filtrado.drop(columns=['_merge'])
df_filtrado=df_filtrado.reset_index()
df_filtrado = df_filtrado.drop(columns=['index'])
df_filtrado

,FECHA,HORA,station,measurement
0,2019-01-01,1,AJM,19.0
1,2019-01-01,2,AJM,17.0
2,2019-01-01,3,AJM,14.0
3,2019-01-01,4,AJM,6.0
4,2019-01-01,5,AJM,4.0
...,...,...,...,...
230697,2020-12-11,20,UIZ,17.0
230698,2020-12-11,21,UIZ,12.0
230699,2020-12-11,22,UIZ,12.0
230700,2020-12-11,23,UIZ,15.0


Una vez con el dataframe inicial limpio, se procede a relacionar cada estación con su respectiva zona de activación. Primeramente se leen los datos

In [27]:
# Cargando dataset de zonas
df_zonas = pd.read_csv('https://raw.githubusercontent.com/IrvingC48/BeduFase2-Proyecto_python/main/Datasets/cat_estacion_depurado.csv')

# Eliminando columna 'Unnamed: 0'
df_zonas = df_zonas.drop(columns=['Unnamed: 0'])
# Remplazando los - por NaN
df_zonas = df_zonas.replace('-', np.nan)
# Eliminando lo NaN's
df_zonas = df_zonas.dropna(axis=0, how='any').reset_index(drop=True)
# Renombrando la columna
df_zonas = df_zonas.rename(columns={'cve_estac': 'station'})
df_zonas.head()

,station,nom_estac,Zona
0,ACO,Acolman,NE
1,AJU,Ajusco,SO
2,AJM,Ajusco Medio,SO
3,ATI,Atizapan,NO
4,BJU,Benito Ju�rez,CE


Se unen los registros con su zona correspondiente

In [28]:
df_filtrado = df_filtrado.merge(df_zonas[['station', 'Zona']], on='station')
df_filtrado.head()

,FECHA,HORA,station,measurement,Zona
0,2019-01-01,1,AJM,19.0,SO
1,2019-01-01,2,AJM,17.0,SO
2,2019-01-01,3,AJM,14.0,SO
3,2019-01-01,4,AJM,6.0,SO
4,2019-01-01,5,AJM,4.0,SO


Con los datos válidos para realizar las operaciones siguientes y ya con la relación de cada estación con su respectiva zona de activación, se procede a calcular el promedio diario de concentración y el índice de calidad del aire en cada zona y en cada día

### Calculando el promedio diario
Para el cálculo del promedio diario de $PM_{2.5}$ se utilizó la metodología descrita en la `NOM-025-SSA1-2014`, siguiendo la siguiente ecuación:

$$\bar{x}= \frac{1}{n} \displaystyle\sum_{i=1}^n x_i$$
$\bar{x}:$ promedio de 24 horas, 

$n:$ número de concentraciones horarias válidas

$x_i:$ concentraciones horarias válidas

- Camo datos extra obteniendo los valores minimos y maximos
- Tambien contabilizando cuantas horas fueron utilizadas para dicho promedio, columna(`N`)

In [29]:
# Calculando promedio diario, contando numero de horas usadas, obteniendo el maximo y minimo
df_PM2524h = df_filtrado.groupby(['FECHA', 'Zona'], as_index=False).agg(PromDiario=('measurement', lambda x: round(x.mean(),1)), #En este caso, se redondea a un decimal
                                                                    Max = ('measurement', 'max'),
                                                                    Min = ('measurement', 'min'))
df_PM2524h.head()

,FECHA,Zona,PromDiario,Max,Min
0,2019-01-01,CE,67.8,165.0,8.0
1,2019-01-01,NE,93.1,393.0,12.0
2,2019-01-01,NO,56.9,162.0,13.0
3,2019-01-01,SE,84.8,180.0,11.0
4,2019-01-01,SO,27.6,97.0,2.0


### Calculando Índice de calidad del aire

Donde:
$$\text{Índice}= (k \times (C \times BP_{Lo}))+I_{Lo}$$

$$k= \frac{I_{Hi}-I_{Lo}}{BP_{Hi}-BP_{Lo}}$$

 
$\text{Índice}:$ Valor del índice para el contaminante deseado.<br>
$C:$ valor redondeado para la concentración del contaminante.<br>
$k:$ constante de proporcionalidad estimada.<br>
$BP_{Hi}:$ valor del punto de corte que es mayor o igual a la concentración a evaluar.<br>
$BP_{Lo}:$ valor del punto de corte que es menor o igual a la concentración a evaluar.<br>
$I_{Hi}:$ valor del índice que corresponde al valor de $BP_{Hi}$<br>
$I_{Lo}:$ valor del índice que corresponde al valor de $BP_{Lo}$<br>

In [30]:
def AQI(df):
    CP = df['PromDiario']
    i = 0

    if 0.0 <= CP <= 12.0:
        k = (50-0)/(12-0)
        i = k * (CP - 0) + 0
    if 12.1 <= CP <= 45.0:
        k = (100-51)/(45-12.1)
        i = k * (CP - 12.1) + 51
    if 45.1 <= CP <= 97.4:
        k = (150-101)/(97.4-45.1)
        i = k * (CP - 45.1) + 101
    if 97.5 <= CP <= 150.4:
        k = (200-151)/(150.4-97.5)
        i = k * (CP - 97.5) + 151
    if 150.5 <= CP <= 250.4:
        k = (300-201)/(250.4-150.5)
        i = k * (CP - 150.5) + 201
    if 250.5 <= CP <= 350.4:
        k = (400-301)/(350.4-250.5)
        i = k * (CP - 250.5) + 301
    if 350.5 <= CP <= 500.4:
        k = (500-401)/(500.4-350.5)
        i = k * (CP - 350.5) + 401

    return round(i, 3)

In [31]:
df_PM2524h['indice'] = df_PM2524h.apply(AQI, axis=1)
df_PM2524h.head()

,FECHA,Zona,PromDiario,Max,Min,indice
0,2019-01-01,CE,67.8,165.0,8.0,122.268
1,2019-01-01,NE,93.1,393.0,12.0,145.971
2,2019-01-01,NO,56.9,162.0,13.0,112.055
3,2019-01-01,SE,84.8,180.0,11.0,138.195
4,2019-01-01,SO,27.6,97.0,2.0,74.085


Con el índice de calidad del aire, se crea otra función para determinar la categoría de calidad del aire descrita por la norma mexicana.

In [32]:
def AQI_categoria(df):
    indice = df['indice']
    if indice <= 50.0:
      clase='Buena'
    elif indice <= 100.0:
      clase='Regular'
    elif indice <= 150.0:
      clase= 'Mala'
    elif indice <= 200.0:
      clase = 'Muy mala'
    else: 
      clase = 'Extremadamente mala'
    
    return clase

Se aplica la función para determinar la categoría a cada promedio diario calculado.

In [33]:
df_PM2524h['clase'] = df_PM2524h.apply(AQI_categoria, axis=1)
df_PM2524h.head()

,FECHA,Zona,PromDiario,Max,Min,indice,clase
0,2019-01-01,CE,67.8,165.0,8.0,122.268,Mala
1,2019-01-01,NE,93.1,393.0,12.0,145.971,Mala
2,2019-01-01,NO,56.9,162.0,13.0,112.055,Mala
3,2019-01-01,SE,84.8,180.0,11.0,138.195,Mala
4,2019-01-01,SO,27.6,97.0,2.0,74.085,Regular


In [ ]:
data_PM2524h.to_csv('../Datasets/PM25/PDEstacion_PM25.csv', index=False)